In [1]:
from stardist.plot import render_label
from csbdeep.utils import normalize
from stardist.models import StarDist2D
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tiff
import skimage.io as io
import skimage.segmentation
import time
import pandas as pd

In [2]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


## DAPI Extraction

Use `DAPI_extraction.ipynb` to get only DAPI intensity signal from spectrally unmixed component data. The output directory from there should be `in_dir` in the code below.

## Nuclear Segmentation 

In [3]:
def StarDist_segmentor(in_dir, save_dir, measure_time_fpath=None):
    '''
    Produces label masks as nuclear segmentation output of StarDist segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields. All images should be in .tiff format
    save_dir_seg: str
        the path to the directory where the segmented images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    times_df_dict = {'img': [], 'time_seconds': []}
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = io.imread(os.path.join(in_dir, file), as_gray=True)
            # loads pretrained stardist model
            model = StarDist2D.from_pretrained('2D_versatile_fluo')
            # predicting nuclei using the StarDist model
            normalized_img = normalize(DAPI_gray)
            start_time = time.time()
            label_mask, _ = model.predict_instances(normalized_img)
            end_time = time.time()
            io.imsave(os.path.join(save_dir, file[:file.find('.tif')] + '.tiff'), label_mask)
            time_taken = end_time - start_time
            times_df_dict['img'].append(file)
            times_df_dict['time_seconds'].append(time_taken)
        else:
            pass

    if measure_time_fpath is not None:
        times_df = pd.DataFrame(times_df_dict)
        times_df.to_csv(measure_time_fpath)
    else:
        pass
        
    return None

## Post Processing 
Post-processing of predicted label masks into binary masks for evaluation. The overall effect of this step is to separate touching nuclei by adding a 2-pixel gap (one-pixel erosion on each nucleus). Next, binary masks (both segmented and ground truth data) can be used to compute the F1-scores by utilizing connected component analysis. 

In [4]:
def postprocess(save_dir_seg, save_dir_bin):
    '''
    Produces binary masks for evaluation from the labelled mask output from the StarDist algorithm

    Parameters
    save_dir_seg: str
        the path to the directory containing the label masks output by the StarDist algorithm with the StarDist_segmentor function. All images should
        be in .tiff format
    save_dir_bin: str
        the path to the directory where the binary images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    # iterating over all files in the input directory
    for file in os.listdir(save_dir_seg):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # Finding the pixels that are touching between any two nuclei
            label_mask = io.imread(os.path.join(save_dir_seg, file))
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir_bin, file[:file.find('.tif')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

## Running the segmentation pipeline

In [5]:
# Replace the strings for in_dir, save_dir_seg and save_dir_bin to paths on the local machine
in_dir = r"D:\nuclear_seg\external_DAPI_tiff"
save_dir_seg = r"D:\nuclear_seg\StarDist_preds\seg"
save_dir_bin = r"D:\nuclear_seg\StarDist_preds\bin"
# To create binary masks using StarDist segmentation algorithm
StarDist_segmentor(in_dir, save_dir_seg, measure_time_fpath=r"D:\nuclear_seg\StarDist_preds\StarDist_times.csv")
postprocess(save_dir_seg,save_dir_bin)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


utils.py (328): D:\nuclear_seg\StarDist_preds\seg\P01-10001(54032.15278)Full_Dapi_Png.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


utils.py (328): D:\nuclear_seg\StarDist_preds\seg\P01-10002(41478.16613)Full_Dapi_Png.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


...

utils.py (328): D:\nuclear_seg\StarDist_preds\seg\P11-10001(52346.6751)Full_Dapi_Png.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


utils.py (328): D:\nuclear_seg\StarDist_preds\seg\P11-10003(58107.9239)Full_Dapi_Png.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


utils.py (328): D:\nuclear_seg\StarDist_preds\seg\P12-10001(43332.8319)Full_Dapi_Png.tiff is a low contrast image
